In [1]:
!pip install autots
# Read in data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import time
import datetime
from sklearn.model_selection import train_test_split
from autots import AutoTS

In [2]:
import warnings

warnings.filterwarnings("ignore")

# Import data

In [3]:
#df

In [4]:
# Define years and months to load data from both 2023 and 2024
years = [2023, 2024]
months = range(1, 13)

# Load datasets dynamically for both years
data_files = [
    f"ig_data-{year}/ig_data_{str(month).zfill(2)}-{year}_u.csv"
    for year in years
    for month in months
]

datasets = [pd.read_csv(file) for file in data_files]

# Add month and year columns
for i, (year, month) in enumerate([(y, m) for y in years for m in months]):
    datasets[i]["month"] = month
    datasets[i]["year"] = year

In [5]:
# Concatenate data
data = pd.concat(datasets)
data

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year
0,2023-01-01,26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...,20558,0,0,0,0,0,0,1,2023
1,2023-01-01,31832168975075c19bfca97103d24b2525b75235b2e61a...,5256,0,0,0,0,0,0,1,2023
2,2023-01-01,065c756ab25ca5147325477d859c320577aa171e55d99b...,337483,1,149,145,4,4101,4511,1,2023
3,2023-01-01,ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...,62556,0,0,0,0,0,0,1,2023
4,2023-01-01,27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...,359455,1,3997,3509,488,57388,63127,1,2023
...,...,...,...,...,...,...,...,...,...,...,...
547988,2024-12-31,8c1ac76cb45374c9710165d154d153ddf48bc30c253c8e...,5905,0,0,0,0,0,0,12,2024
547989,2024-12-31,834715aec99b1e9be4feccadeea4712f1cde42d649664d...,211245,1,345,337,8,10157,11172,12,2024
547990,2024-12-31,94528532ec6bee015266fdaeb9cd570a6fbf6e3a2dfbc2...,1660,0,0,0,0,0,0,12,2024
547991,2024-12-31,0597c04c318d668379c1d549c31931c56838f6865ae7f3...,10084,1,28,28,0,2914,3205,12,2024


In [6]:
# find in the column followers the rows that have the value as "undefined"
data[data["followers"] == "undefined"]
# replace the "undefined" in follwoers with nan
data["followers_num"] = data["followers"].replace("undefined", np.nan)
# convert the followers_num to integer
data["followers_num"] = data["followers_num"].astype(float).astype(pd.Int64Dtype())
# Ensure followers_num is numeric and clean
data["followers_num"] = pd.to_numeric(data["followers_num"], errors="coerce")

# Step 4: Replace zeros with NaN (to avoid filling with zero)
data["followers_num"].replace(0, np.nan, inplace=True)

# Step 5: Fill NaN values using the last non-null value, making sure it isn't zero
data["followers_num"] = data["followers_num"].fillna(method="ffill")

# Step 6: If still NaN (if there were no valid previous non-null value), you can set to zero or some other value
data["followers_num"].fillna(0, inplace=True)

# Step 7: Ensure the column type is integer, but with nullable integer type to handle missing values
data["followers_num"] = data["followers_num"].astype(pd.Int64Dtype())

# Verify the changes
print(data.head())


         date                                         profile_id followers  \
0  2023-01-01  26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...     20558   
1  2023-01-01  31832168975075c19bfca97103d24b2525b75235b2e61a...      5256   
2  2023-01-01  065c756ab25ca5147325477d859c320577aa171e55d99b...    337483   
3  2023-01-01  ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...     62556   
4  2023-01-01  27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...    359455   

   posts  engagement  likes  comments  reach  impressions  month  year  \
0      0           0      0         0      0            0      1  2023   
1      0           0      0         0      0            0      1  2023   
2      1         149    145         4   4101         4511      1  2023   
3      0           0      0         0      0            0      1  2023   
4      1        3997   3509       488  57388        63127      1  2023   

   followers_num  
0          20558  
1           5256  
2         337483  
3         

In [7]:
data.isnull().sum()

date             0
profile_id       0
followers        0
posts            0
engagement       0
likes            0
comments         0
reach            0
impressions      0
month            0
year             0
followers_num    0
dtype: int64

In [8]:
data[data["followers_num"] == "undefined"]

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year,followers_num


In [9]:
profile_counts = data["profile_id"].value_counts()
# # get the ids of the accounts that have more than 365 records
# more_than_299_ids = profile_counts[profile_counts >= 300].index
# # filter the data to include only the accounts that have more than 365 records
# more_than_299_data = data[data["profile_id"].isin(more_than_299_ids)]
# get the ids of the accounts that have more than 365 records
more_than_365_ids = profile_counts[profile_counts > 365].index
# filter the data to include only the accounts that have more than 365 records
more_than_365_data = data[data["profile_id"].isin(more_than_365_ids)]
# get the ids of the accounts that have 300 to 365 records
between_365_300_ids = profile_counts[
    (profile_counts <= 365) & (profile_counts >= 300)
].index
# filter the data to include only the accounts that have 300 to 365 records
between_365_300_data = data[data["profile_id"].isin(between_365_300_ids)]
# get the ids of the accounts that have 90 to 299 records
between_299_90_ids = profile_counts[
    (profile_counts < 300) & (profile_counts >= 90)
].index
# filter the data to include only the accounts that have 90 to 299 records
between_299_90_data = data[data["profile_id"].isin(between_299_90_ids)]

In [12]:
# Export more than 365 data to csv
between_365_300_data.to_csv("between_365_300_data.csv", index=False)

# Export more than 365 data to csv
between_299_90_data.to_csv("between_299_90_data.csv", index=False)

## 1.baseline model

In [78]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Create a Linear Regression model for each profile_id
predictions = {}
for profile_id in train_df.columns:
    # Prepare features (time in months)
    X_train = np.arange(len(train_df)).reshape(-1, 1)
    y_train = train_df[profile_id].values

    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict for test period
    X_test = np.arange(len(train_df), len(train_df) + len(test_df)).reshape(-1, 1)
    y_pred = model.predict(X_test)
    
    predictions[profile_id] = y_pred

# Convert predictions to DataFrame
forecast_df = pd.DataFrame(predictions, index=test_df.index)

# Evaluate predictions
for profile_id in test_df.columns:
    true_values = test_df[profile_id]
    predicted_values = forecast_df[profile_id]
    
    mae = mean_absolute_error(true_values, predicted_values)
    mse = mean_squared_error(true_values, predicted_values)
    
    print(f"Profile {profile_id}: MAE = {mae:.2f}, MSE = {mse:.2f}")


Profile 013fb9bb39ff5ff33b90047ec453d42d9640edba9f31315082b61d7c4128f24c: MAE = 10150.38, MSE = 145050378.01
Profile 077da2e189fb7162816894d2e595d574542e946fcbbe1d97b3c1b74a8aec251a: MAE = 3614.23, MSE = 13161976.57
Profile 0be69c62c8641f50ad9e453fe1ad6b9cc93c2a0254bb9c98939bae221bd4f8d7: MAE = 586094.44, MSE = 344762964870.95
Profile 0c4321e97dc4002a45ef8cef7eaa46b278dbb2bdc7ae7fd52f11a08dfa02d440: MAE = 17428.97, MSE = 322522174.16
Profile 164ef4e9851a47c613110541d0eb6fa142f6893c6fb1549150022652bde4701e: MAE = 5243389.83, MSE = 29294242006350.39
Profile 1e74d68ae1fd02b5fd7810b57159a95d1311c21b55fa3ac57a5cab3fa94beedc: MAE = 40869.04, MSE = 1673606896.83
Profile 1ff3c48eb005e419c575d215f9daeaa4a8b6bae35ab8ef3d44cfa663cbbaecd1: MAE = 102970.90, MSE = 11397455242.65
Profile 2ad8a04784032dad9d7f77dea27f070f593e6f2e11f2733e9b5d484c043fbdd0: MAE = 172398.66, MSE = 31348454424.42
Profile 2e3fb95aa0750a21f395d2eb330ca63d70cac36ba30105d04f3515d2474e48d2: MAE = 1147615.15, MSE = 1390128522196.

In [80]:
# Loop through each profile_id for Linear Regression Model
# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

results_lr = {}

for profile_id in profile_ids:
    # Step 1: Calculate the confidence interval for true follower numbers (followers_num)
    mean_followers = test_df_aligned[profile_id].mean()
    std_followers = test_df_aligned[profile_id].std()

    # Calculate the 95% confidence interval for true followers
    confidence_interval_lower = mean_followers - 1.96 * (std_followers / np.sqrt(len(test_df_aligned[profile_id])))
    confidence_interval_upper = mean_followers + 1.96 * (std_followers / np.sqrt(len(test_df_aligned[profile_id])))


    # Step 3: Check how many linear regression predictions fall within the confidence interval
    predictions_within_interval_lr = (
        (forecast_df[profile_id] >= confidence_interval_lower) & 
        (forecast_df[profile_id] <= confidence_interval_upper)
    )

    # Step 4: Calculate the number of predictions within the confidence interval
    num_predictions_within_interval_lr = len(predictions_within_interval_lr)

    # Step 5: Calculate the ratio of linear regression predictions within the confidence interval
    ratio_within_interval_lr = num_predictions_within_interval_lr / len(test_df_aligned[profile_id])

    # Store results for this profile_id for Linear Regression
    results_lr[profile_id] = {
        "confidence_interval_lower": confidence_interval_lower,
        "confidence_interval_upper": confidence_interval_upper,
        "num_predictions_within_interval_lr": num_predictions_within_interval_lr,
        "ratio_within_interval_lr": ratio_within_interval_lr,
    }

# Convert results for Linear Regression to a DataFrame for better visualization
results_df_lr = pd.DataFrame(results_lr).T
results_df_lr


,confidence_interval_lower,confidence_interval_upper,num_predictions_within_interval_lr,ratio_within_interval_lr
013fb9bb39ff5ff33b90047ec453d42d9640edba9f31315082b61d7c4128f24c,6.944000e+03,6.944000e+03,6.0,1.0
077da2e189fb7162816894d2e595d574542e946fcbbe1d97b3c1b74a8aec251a,7.340744e+05,7.413436e+05,6.0,1.0
0be69c62c8641f50ad9e453fe1ad6b9cc93c2a0254bb9c98939bae221bd4f8d7,2.779000e+03,2.779000e+03,6.0,1.0
0c4321e97dc4002a45ef8cef7eaa46b278dbb2bdc7ae7fd52f11a08dfa02d440,1.104265e+03,1.107068e+03,6.0,1.0
164ef4e9851a47c613110541d0eb6fa142f6893c6fb1549150022652bde4701e,1.627173e+07,1.642589e+07,6.0,1.0
1e74d68ae1fd02b5fd7810b57159a95d1311c21b55fa3ac57a5cab3fa94beedc,1.139100e+04,1.139100e+04,6.0,1.0
1ff3c48eb005e419c575d215f9daeaa4a8b6bae35ab8ef3d44cfa663cbbaecd1,2.405000e+03,2.405000e+03,6.0,1.0
2ad8a04784032dad9d7f77dea27f070f593e6f2e11f2733e9b5d484c043fbdd0,3.734000e+04,3.734000e+04,6.0,1.0
2e3fb95aa0750a21f395d2eb330ca63d70cac36ba30105d04f3515d2474e48d2,5.163500e+04,5.163500e+04,6.0,1.0
329f0aac45869b17ef7b28903bbfd2f0510ed60c31751770d663a6c6072ab551,2.113200e+04,2.113200e+04,6.0,1.0


## 2. AutoTS

In [14]:
import pandas as pd
import numpy as np
from autots import AutoTS
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Initialize and train the AutoTS model (without 'fillna' argument)
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='M',
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate predictions
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    
    mae = mean_absolute_error(true_values, predicted_values)
    mse = mean_squared_error(true_values, predicted_values)
    print(f"Profile {profile_id}: MAE = {mae}, MSE = {mse}")

Using 4 cpus for n_jobs.
Data frequency is: M, used frequency is: M
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Template Eval Error: OSError(12, 'Cannot allocate memory') in model 5 in generation 0: DatepartRegression
Model Number: 6 with model DatepartRegression in generation 0 of 10


--------------------------------------------------------------------------------
LokyProcess-1 failed with traceback: 
--------------------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/joblib/externals/l

In [17]:
import joblib

# Suppose 'model' is your trained AutoTS model
joblib.dump(model, 'autoTS_model_monthly.pkl')
print("Model saved successfully!")

Model saved successfully!


In [57]:
print("test_df_aligned shape:", test_df_aligned.shape)
print("forecast_df shape:", forecast_df.shape)

test_df_aligned shape: (6, 47)
forecast_df shape: (6, 60)


In [58]:
common_index = test_df_aligned.index.intersection(forecast_df.index)  # Get matching indices
test_df_aligned = test_df_aligned.loc[common_index]
forecast_df = forecast_df.loc[common_index]
forecast_df = forecast_df[test_df_aligned.columns]  # Keep only matching columns
print("test_df_aligned shape:", test_df_aligned.shape)
print("forecast_df shape:", forecast_df.shape)

test_df_aligned shape: (6, 47)
forecast_df shape: (6, 47)


In [59]:
import numpy as np
import pandas as pd

profile_ids = test_df.columns

# Initialize a dictionary to store results for each profile_id
results = {}

# Loop through each profile_id
for profile_id in profile_ids:
    # Step 1: Calculate the confidence interval for true follower numbers (followers_num)
    mean_followers = test_df_aligned[profile_id].mean()
    std_followers = test_df_aligned[profile_id].std()

    # Calculate the 95% confidence interval for true followers
    confidence_interval_lower = mean_followers - 1.96 * (std_followers / np.sqrt(len(test_df_aligned[profile_id])))
    confidence_interval_upper = mean_followers + 1.96 * (std_followers / np.sqrt(len(test_df_aligned[profile_id])))

    # Step 2: Check how many predictions fall within the confidence interval
    predictions_within_interval = (
        (forecast_df[profile_id] >= confidence_interval_lower) & 
        (forecast_df[profile_id] <= confidence_interval_upper)
    )

    # Step 3: Calculate the number of predictions within the confidence interval
    num_predictions_within_interval = len(predictions_within_interval)

    # Step 4: Calculate the ratio of predictions within the confidence interval
    ratio_within_interval = num_predictions_within_interval / len(test_df_aligned[profile_id])

    # Store results for this profile_id
    results[profile_id] = {
        "confidence_interval_lower": confidence_interval_lower,
        "confidence_interval_upper": confidence_interval_upper,
        "num_predictions_within_interval": num_predictions_within_interval,
        "ratio_within_interval": ratio_within_interval,
    }

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
results_df

,confidence_interval_lower,confidence_interval_upper,num_predictions_within_interval,ratio_within_interval
013fb9bb39ff5ff33b90047ec453d42d9640edba9f31315082b61d7c4128f24c,6.944000e+03,6.944000e+03,6.0,1.0
077da2e189fb7162816894d2e595d574542e946fcbbe1d97b3c1b74a8aec251a,7.340744e+05,7.413436e+05,6.0,1.0
0be69c62c8641f50ad9e453fe1ad6b9cc93c2a0254bb9c98939bae221bd4f8d7,2.779000e+03,2.779000e+03,6.0,1.0
0c4321e97dc4002a45ef8cef7eaa46b278dbb2bdc7ae7fd52f11a08dfa02d440,1.104265e+03,1.107068e+03,6.0,1.0
164ef4e9851a47c613110541d0eb6fa142f6893c6fb1549150022652bde4701e,1.627173e+07,1.642589e+07,6.0,1.0
1e74d68ae1fd02b5fd7810b57159a95d1311c21b55fa3ac57a5cab3fa94beedc,1.139100e+04,1.139100e+04,6.0,1.0
1ff3c48eb005e419c575d215f9daeaa4a8b6bae35ab8ef3d44cfa663cbbaecd1,2.405000e+03,2.405000e+03,6.0,1.0
2ad8a04784032dad9d7f77dea27f070f593e6f2e11f2733e9b5d484c043fbdd0,3.734000e+04,3.734000e+04,6.0,1.0
2e3fb95aa0750a21f395d2eb330ca63d70cac36ba30105d04f3515d2474e48d2,5.163500e+04,5.163500e+04,6.0,1.0
329f0aac45869b17ef7b28903bbfd2f0510ed60c31751770d663a6c6072ab551,2.113200e+04,2.113200e+04,6.0,1.0


In [75]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Store evaluation results
evaluation_results = []

# ----- LINEAR REGRESSION MODEL -----
predictions_lr = {}
for profile_id in train_df.columns:
    X_train = np.arange(len(train_df)).reshape(-1, 1)
    y_train = train_df[profile_id].values

    model = LinearRegression()
    model.fit(X_train, y_train)

    X_test = np.arange(len(train_df), len(train_df) + len(test_df)).reshape(-1, 1)
    y_pred = model.predict(X_test)
    predictions_lr[profile_id] = y_pred

forecast_df_lr = pd.DataFrame(predictions_lr, index=test_df.index)

# Evaluate Linear Regression model
for profile_id in test_df.columns:
    mae_lr = mean_absolute_error(test_df[profile_id], forecast_df_lr[profile_id])
    mse_lr = mean_squared_error(test_df[profile_id], forecast_df_lr[profile_id])
    evaluation_results.append({'Profile': profile_id, 'Model': 'Linear Regression', 'MAE_LR': mae_lr, 'MSE_LR': mse_lr})

# ----- AUTOTS MODEL -----
from autots import AutoTS

# Initialize and train the AutoTS model (without 'fillna' argument)
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='M',
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results.append({'Profile': profile_id, 'Model': 'AutoTS', 'MAE_AutoTS': mae_autots, 'MSE_AutoTS': mse_autots})

# ----- CREATE COMPARISON DATAFRAME -----
#evaluation_df = pd.DataFrame(evaluation_results)
evaluation_results


Using 4 cpus for n_jobs.
Data frequency is: M, used frequency is: M
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py

Model Number: 6 with model DatepartRegression in generation 0 of 10
Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10
Model Number: 11 with model GLM in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Template Eval Error: Exception("Transformer PowerTransformer failed on fit from params rolling_mean_24 {'0': {}, '1': {'model': 'middle', 'decimals': 2, 'on_transform': False, 'on_inverse': True}, '2': {}} with error BracketError('The algorithm terminated without finding a valid bracket. Consider trying different initial points.')") in model 18 in generation 0: SeasonalNaive
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number: 21 with model VAR in generation 0 of 10
Template

...

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter al

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1430, in model_forecast
    df_forecast = model_forecast(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1512, in model_forecast
    raise ValueError(
ValueError: Model Ensemble returned NaN for one or more series. fail_on_forecast_nan=True

FAILED: Ensemble BestN component 2 of 3 Ensemble with error: ValueError('Model Ensemble returned NaN for one or more series. fail_on_forecast_nan=True')
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1430, in model_forecast
    df_forecast = model_forecast(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1512, in model_forecast
    raise ValueError(
ValueError: Model Ensemble

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter al

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1430, in model_forecast
    df_forecast = model_forecast(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1512, in model_forecast
    raise ValueError(
ValueError: Model Ensemble returned NaN for one or more series. fail_on_forecast_nan=True

FAILED: Ensemble BestN component 2 of 5 Ensemble with error: ValueError('Model Ensemble returned NaN for one or more series. fail_on_forecast_nan=True')
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1430, in model_forecast
    df_forecast = model_forecast(
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/autots/evaluator/auto_model.py", line 1512, in model_forecast
    raise ValueError(
ValueError: Model Ensemble

[{'Profile': '013fb9bb39ff5ff33b90047ec453d42d9640edba9f31315082b61d7c4128f24c',
  'Model': 'Linear Regression',
  'MAE_LR': 10150.380002293323,
  'MSE_LR': 145050378.01333818},
 {'Profile': '077da2e189fb7162816894d2e595d574542e946fcbbe1d97b3c1b74a8aec251a',
  'Model': 'Linear Regression',
  'MAE_LR': 3614.2289301684746,
  'MSE_LR': 13161976.565487152},
 {'Profile': '0be69c62c8641f50ad9e453fe1ad6b9cc93c2a0254bb9c98939bae221bd4f8d7',
  'Model': 'Linear Regression',
  'MAE_LR': 586094.4389404885,
  'MSE_LR': 344762964870.94684},
 {'Profile': '0c4321e97dc4002a45ef8cef7eaa46b278dbb2bdc7ae7fd52f11a08dfa02d440',
  'Model': 'Linear Regression',
  'MAE_LR': 17428.967836257325,
  'MSE_LR': 322522174.1571599},
 {'Profile': '164ef4e9851a47c613110541d0eb6fa142f6893c6fb1549150022652bde4701e',
  'Model': 'Linear Regression',
  'MAE_LR': 5243389.826281388,
  'MSE_LR': 29294242006350.395},
 {'Profile': '1e74d68ae1fd02b5fd7810b57159a95d1311c21b55fa3ac57a5cab3fa94beedc',
  'Model': 'Linear Regression',


In [76]:
# Convert to DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

# Now pivot the DataFrame to have both models' MAE and MSE in the same row for each profile
evaluation_df_combined = evaluation_df.pivot_table(index='Profile', 
                                                   columns='Model', 
                                                   values=['MAE_LR', 'MSE_LR', 'MAE_AutoTS', 'MSE_AutoTS'], 
                                                   aggfunc='first')

# Flatten multi-level columns for easy viewing
evaluation_df_combined.columns = ['MAE_LR', 'MSE_LR', 'MAE_AutoTS', 'MSE_AutoTS']

# Print the comparison DataFrame
evaluation_df_combined

,MAE_LR,MSE_LR,MAE_AutoTS,MSE_AutoTS
Profile,,,,
013fb9bb39ff5ff33b90047ec453d42d9640edba9f31315082b61d7c4128f24c,6.944000e+03,1.015038e+04,4.821914e+07,1.450504e+08
077da2e189fb7162816894d2e595d574542e946fcbbe1d97b3c1b74a8aec251a,7.377090e+05,3.614229e+03,5.442318e+11,1.316198e+07
0be69c62c8641f50ad9e453fe1ad6b9cc93c2a0254bb9c98939bae221bd4f8d7,2.779000e+03,5.860944e+05,7.722841e+06,3.447630e+11
0c4321e97dc4002a45ef8cef7eaa46b278dbb2bdc7ae7fd52f11a08dfa02d440,1.105667e+03,1.742897e+04,1.222501e+06,3.225222e+08
164ef4e9851a47c613110541d0eb6fa142f6893c6fb1549150022652bde4701e,1.634881e+07,5.243390e+06,2.672913e+14,2.929424e+13
1e74d68ae1fd02b5fd7810b57159a95d1311c21b55fa3ac57a5cab3fa94beedc,1.139100e+04,4.086904e+04,1.297549e+08,1.673607e+09
1ff3c48eb005e419c575d215f9daeaa4a8b6bae35ab8ef3d44cfa663cbbaecd1,2.405000e+03,1.029709e+05,5.784025e+06,1.139746e+10
2ad8a04784032dad9d7f77dea27f070f593e6f2e11f2733e9b5d484c043fbdd0,3.734000e+04,1.723987e+05,1.394276e+09,3.134845e+10
2e3fb95aa0750a21f395d2eb330ca63d70cac36ba30105d04f3515d2474e48d2,5.163500e+04,1.147615e+06,2.666173e+09,1.390129e+12
